#  Predicting Cadets Likely to Struggle in Physics I

While the United States Coast Guard Academy (USCGA) recruits high-achieving students from across the nation and around the globe, every year we have cadets who struggle to pass Physics I.  Failure to pass Physics has many undesirable consequences, including lowering a cadet's GPA, potentially delaying graduation and deployment to the operational fleet, and possibly even leading to a cadet's disenrollment from the Academy.  It is in the best interests of both the United States Coast Guard and the cadets themselves to do whatever we can to help cadets to pass Physics I the first time they take it.  Early intervention is vital, and academic support programs are in place to help struggling cadets.  However, at-risk cadets are difficult to identify until they have already failed an exam or two, at which point they are already in distress.  This tool seeks to mitigate that problem by predicting - right at the start of the semester - the cadets most at risk to have an unfavorable outcome in Physics I (defined here as earning a final grade of C- or worse), for the purpose of getting them into an academic support program at the earliest sign of trouble.

## Model Training

This tool is a ML model that identifies cadets most at risk to struggle in Physics I based on their prior performance at the USCGA and in high school.  Both the training and prediction data require access to cadet PII, including grades and GPAs.  Consequently, the tool is presented as a Jupyter notebook for use by the USCGA Vice Provost for Academic Affairs or their designee (e.g. the Physics I Course Coordinator).

In it's current form, the model has been trained on cadet data for Class Years 2020-2025.  You should not retrain the model unless you wish to update it based on the performance of classes since then.  If you only want to identify at-risk cadets fot the upcoming semester, proceed to the "Predictions" section.

### Import and merge training data, import target data

First, we'll import the institutional training data.  The current code assumes the cadet data will be presented as a .csv file with 15 fields in each row. The first field is a unique identifier (i.e. Cadet Code or equivalent).  The other fields and their data types are:
* Gender (string)
* Academic Class Year  (integer)  
* Ethnicity (string)
* Math Placement (integer, CGA math class number) 
* Most recent CGA GPA (decimal)
* HS GPA (decimal)
* HS GPA Scale (4.0, 100, etc.) (decimal)
* Standardized Test Used (SAT or ACT) (string) 
* Math Score (SAT equivalent) (integer)  
* Verbal Score (SAT equivalent) (integer)  
* Times enrolled in Calculus I (integer) 
* Calculus 1 Last Grade (letter) 
* Calculus I Last Quality Points (decimal)
* Times enrolled in Physics I (integer, 1 unless repeating)  

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [2]:
import numpy as np
import pandas as pd

train_instit = pd.read_csv("./Train-instit.csv", index_col=0)

print(type(train_instit))
train_instit.shape

<class 'pandas.core.frame.DataFrame'>


(1684, 14)

In [3]:
train_instit.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1684 entries, 1 to 1684
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Gender           1684 non-null   object 
 1   AcademicClassYr  1684 non-null   int64  
 2   Ethnicity        1671 non-null   object 
 3   MathPlacement    1684 non-null   int64  
 4   LastGPA          1679 non-null   float64
 5   HSGPA            1684 non-null   float64
 6   MaxHS_GPA_Scale  1684 non-null   float64
 7   TestUsed         1684 non-null   object 
 8   MaxMath          1684 non-null   int64  
 9   MaxVerbal        1684 non-null   int64  
 10  Calc1Number      1684 non-null   int64  
 11  Calc1LastGrade   1364 non-null   object 
 12  Calc1LastQP      1348 non-null   float64
 13  PhysicsITimes    1684 non-null   int64  
dtypes: float64(4), int64(6), object(4)
memory usage: 197.3+ KB


The provided institutional data includes the demographic information "Gender" and "Ethnicity".  We do not want these categories to bias our results, so we'll drop them.  We'll also drop the Class Year (not predictive) and "PhysicsITimes", as we'll bring that in with the departmental data.

In [4]:
train_instit.drop(columns=['Gender', 'AcademicClassYr', 'Ethnicity', 'PhysicsITimes'], inplace=True)

In [5]:
train_instit['MathPlacement'] = train_instit['MathPlacement'].astype(str) #Set MathPlacement (class number) as string

Now we'll import departmental data.  This also starts with a unique identifier for each cadet that matches those in the institutional data.  The other fields are "Phys1Grade" (a cadet's Physics 1 final grade for each attempt) and "Physics1Times".  Note that REGIS (the CGA academic record system) may provide only the most recent value for Physics1Times, so the data for cadets repeating Physics 1 in any semester had to be manually updated to match attempt instances with their final grades for each attempt.  Future updates to the training data should be ssimilarly pre-processed.

In [6]:
train_dept = pd.read_csv("./Train-dept-v2.csv", index_col=0)

print(type(train_dept))
train_dept.shape

<class 'pandas.core.frame.DataFrame'>


(1702, 3)

In [7]:
train_df=train_dept.merge(train_instit, on='RecordNum', how='left')

In [8]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1702 entries, 1 to 1457
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Phys1Grade       1563 non-null   object 
 1   PhysicsITimes    1702 non-null   int64  
 2   P1_PrevGrade     1702 non-null   object 
 3   MathPlacement    1702 non-null   object 
 4   LastGPA          1697 non-null   float64
 5   HSGPA            1702 non-null   float64
 6   MaxHS_GPA_Scale  1702 non-null   float64
 7   TestUsed         1702 non-null   object 
 8   MaxMath          1702 non-null   int64  
 9   MaxVerbal        1702 non-null   int64  
 10  Calc1Number      1702 non-null   int64  
 11  Calc1LastGrade   1381 non-null   object 
 12  Calc1LastQP      1365 non-null   float64
dtypes: float64(4), int64(4), object(5)
memory usage: 186.2+ KB


Let's find the mean of the standardized test scores "MaxMath" and "MaxVerbal" in case we need to fill any missing values in future cohorts.

In [9]:
SAT_Math_avg = int(train_df.MaxMath.mean())
SAT_Verbal_avg = int(train_df.MaxVerbal.mean())
print(SAT_Math_avg, SAT_Verbal_avg)

655 652


### Data Cleaning

We dropped drop the 'Gender' and 'Ethnicity' columns during data import, on the grounds that these may bias our results.  We also dropped the "AcademicClassYr" column, since year-on-year performance should be roughly similar.  We dropped them from the prediction set as well.  We'll use "Calc1QP" as the Calc 1 performance indicator, so we'll drop "Calc1LastGrade" too.

In [10]:
train_inst=train_df.drop(columns=['Calc1LastGrade'])

In [11]:
train_inst.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1702 entries, 1 to 1457
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Phys1Grade       1563 non-null   object 
 1   PhysicsITimes    1702 non-null   int64  
 2   P1_PrevGrade     1702 non-null   object 
 3   MathPlacement    1702 non-null   object 
 4   LastGPA          1697 non-null   float64
 5   HSGPA            1702 non-null   float64
 6   MaxHS_GPA_Scale  1702 non-null   float64
 7   TestUsed         1702 non-null   object 
 8   MaxMath          1702 non-null   int64  
 9   MaxVerbal        1702 non-null   int64  
 10  Calc1Number      1702 non-null   int64  
 11  Calc1LastQP      1365 non-null   float64
dtypes: float64(4), int64(4), object(4)
memory usage: 172.9+ KB


There appear to be 5 missing values in the "LastGPA" column.  These are likely due to disenrollments at the beginning of the semester, and should be dropped.

In [12]:
train_inst.dropna(subset=['LastGPA'], inplace=True)

About 300 records are missing values for 'Calc1LastGrade' and 'Calc1LastQP'.  This could be because their math placement exam placed them into a class higher than Calc 1, or because they hadn't yet taken Calc 1.  If the former, their effective Calc 1 grade should be 'A' and their Calc 1 Quality Points should be 4.0.  For those cadets, their effective Calc1Number should also be set to 1.  For the others, their QP should be 0.0.

In [13]:
train_inst['Calc1LastQP'] = train_inst.apply(lambda row: 
                                             0.0 
                                             if pd.isnull(row['Calc1LastQP']) and int(row['MathPlacement']) <= 3111
                                             else row['Calc1LastQP'], axis=1)

train_inst['Calc1LastQP'] = train_inst.apply(lambda row: 
                                             4.0 
                                             if pd.isnull(row['Calc1LastQP']) and int(row['MathPlacement']) > 3111
                                             else row['Calc1LastQP'], axis=1)

train_inst['Calc1Number'] = train_inst.apply(lambda row: 
                                             1 
                                             if int(row['MathPlacement']) > 3111
                                             else row['Calc1Number'], axis=1)

Let's get the HS GPAs on the same scale.

In [14]:
train_inst['HS_Gpct'] = train_inst.HSGPA / train_inst.MaxHS_GPA_Scale

In [15]:
train_inst.HS_Gpct.replace([np.inf, -np.inf], np.nan, inplace=True)  #taking care of some pesky inf values

About 300 cadets in our training data don't have HS GPAs listed!  For now, we'll fill them with their Academy GPA.

In [16]:
train_inst['HS_Gpct'] = train_inst.apply(lambda row: 
                                             row['LastGPA']/4.0 
                                             if pd.isnull(row['HS_Gpct'])
                                             else row['HS_Gpct'], axis=1)

Finally, we need to handle the Physics 1 grades (the thing we're predicting).

Many Physics 1 grades (the target of our predictions) are missing from the training data.  In many cases, these are due to "validation": i.e., testing out of Physics 1.  These cadets almost universally placed into Calc 1 or higher.  For now, we'll assume that any missing Physics 1 grade where the cadet was placed into MATH 3211 was a validation.

In [17]:
train_inst['Phys1Grade'] = train_inst.apply(lambda row: 
                                             'V' 
                                             if pd.isnull(row['Phys1Grade']) and int(row['MathPlacement']) >= 3111
                                             else row['Phys1Grade'], axis=1)

In [18]:
train_inst.Phys1Grade.value_counts()

Phys1Grade
B+    280
A     272
B     198
C+    163
B-    151
A-    146
V     127
H     121
C     110
C-     51
F      36
D      20
W      14
S       1
Name: count, dtype: int64

In [19]:
train_inst.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1697 entries, 1 to 1457
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Phys1Grade       1690 non-null   object 
 1   PhysicsITimes    1697 non-null   int64  
 2   P1_PrevGrade     1697 non-null   object 
 3   MathPlacement    1697 non-null   object 
 4   LastGPA          1697 non-null   float64
 5   HSGPA            1697 non-null   float64
 6   MaxHS_GPA_Scale  1697 non-null   float64
 7   TestUsed         1697 non-null   object 
 8   MaxMath          1697 non-null   int64  
 9   MaxVerbal        1697 non-null   int64  
 10  Calc1Number      1697 non-null   int64  
 11  Calc1LastQP      1697 non-null   float64
 12  HS_Gpct          1697 non-null   float64
dtypes: float64(5), int64(4), object(4)
memory usage: 185.6+ KB


Grades of W or S are usually due to extenuating circumstances.  We'll drop them, along with the 7 rows still missing grades.

In [20]:
valid_grades = ['V', 'H', 'A', 'A-', 'B+', 'B', 'B-', 'C+', 'C', 'C-', 'D', 'F']

In [21]:
df = train_inst[train_inst.Phys1Grade.isin(valid_grades) == True]

In [22]:
df.Phys1Grade.value_counts()

Phys1Grade
B+    280
A     272
B     198
C+    163
B-    151
A-    146
V     127
H     121
C     110
C-     51
F      36
D      20
Name: count, dtype: int64

Note that the classes here are HIGHLY unbalanced.  Only about 1 in 15 cadets earns a grade of C- or worse.  We will beed to address this in the model training section.

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1675 entries, 1 to 1457
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Phys1Grade       1675 non-null   object 
 1   PhysicsITimes    1675 non-null   int64  
 2   P1_PrevGrade     1675 non-null   object 
 3   MathPlacement    1675 non-null   object 
 4   LastGPA          1675 non-null   float64
 5   HSGPA            1675 non-null   float64
 6   MaxHS_GPA_Scale  1675 non-null   float64
 7   TestUsed         1675 non-null   object 
 8   MaxMath          1675 non-null   int64  
 9   MaxVerbal        1675 non-null   int64  
 10  Calc1Number      1675 non-null   int64  
 11  Calc1LastQP      1675 non-null   float64
 12  HS_Gpct          1675 non-null   float64
dtypes: float64(5), int64(4), object(4)
memory usage: 183.2+ KB


As a last step, we will convert our Physics 1 grades into our target labels.  Grades of C- or higher are "successful", while those below are not.

In [24]:
#Successful_grades = ['V', 'H', 'A', 'A-', 'B+', 'B', 'B-', 'C+', 'C']  #moved C- to the At Risk category to help balance classes

In [25]:
def GradeConverter(row):
    successful_grades = ['V', 'H', 'A', 'A-', 'B+', 'B', 'B-', 'C+', 'C']
    
    if row['Phys1Grade'] in successful_grades:
        outcome = 0
    else:
        outcome = 1
        
    return outcome    

In [26]:
df['Phys1_outcome'] = df.apply(lambda row: GradeConverter(row), axis=1)

/tmp/ipykernel_25759/3163381256.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Phys1_outcome'] = df.apply(lambda row: GradeConverter(row), axis=1)


In [27]:
df.Phys1_outcome.value_counts()

Phys1_outcome
0    1568
1     107
Name: count, dtype: int64

In [28]:
model_input = df.copy()

In [29]:
features_to_use = ['MathPlacement','HS_Gpct', 'MaxMath', 'MaxVerbal', 'LastGPA', 'Calc1Number', 'Calc1LastQP', 'PhysicsITimes', 'P1_PrevGrade']

X_m = model_input[features_to_use]
y_m = model_input['Phys1_outcome']

In [30]:
X_m.head()

,MathPlacement,HS_Gpct,MaxMath,MaxVerbal,LastGPA,Calc1Number,Calc1LastQP,PhysicsITimes,P1_PrevGrade
RecordNum,,,,,,,,,
1,3111,0.9500,630,630,2.98,1,2.3,1,N
2,3111,0.8988,800,580,3.06,1,3.0,1,N
3,3111,0.8150,550,600,3.26,1,3.0,1,N
4,3111,0.7525,630,540,2.60,1,1.0,1,N
5,3111,0.9400,680,600,3.52,1,3.0,1,N


In [31]:
y_m.head()

RecordNum
1    0
2    0
3    0
4    0
5    0
Name: Phys1_outcome, dtype: int64

Now we'll encode the columns in preparation for sampling and classification

We'll need to select and transform the columns we want to feed into our predictor.  MathPlacement is categorical and will need to be OneHotEncoded.  We'll also keep HS_Gpct, LastGPA, MaxMath, MaxVerbal, Calc1Number, Calc1LastQP, and PhysicsITimes.  Phys1_outcome is what we'll be trying to predict.

In [32]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

columns_to_encode = ['MathPlacement', 'P1_PrevGrade']
ohe = OneHotEncoder()

ohe_trans = ColumnTransformer([
    ('OHE', ohe, columns_to_encode)], 
    remainder='passthrough')

X_encoded = ohe_trans.fit_transform(X_m)
Xenc_cols = ohe_trans.get_feature_names_out()

In [33]:
X_encoded.shape

(1675, 16)

In [34]:
Xenc_cols

array(['OHE__MathPlacement_3107', 'OHE__MathPlacement_3111',
       'OHE__MathPlacement_3115', 'OHE__MathPlacement_3117',
       'OHE__MathPlacement_3211', 'OHE__MathPlacement_3213',
       'OHE__MathPlacement_3215', 'OHE__P1_PrevGrade_F',
       'OHE__P1_PrevGrade_N', 'remainder__HS_Gpct', 'remainder__MaxMath',
       'remainder__MaxVerbal', 'remainder__LastGPA',
       'remainder__Calc1Number', 'remainder__Calc1LastQP',
       'remainder__PhysicsITimes'], dtype=object)

In [35]:
X=pd.DataFrame(X_encoded, columns=Xenc_cols)

In [36]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1675 entries, 0 to 1674
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   OHE__MathPlacement_3107   1675 non-null   float64
 1   OHE__MathPlacement_3111   1675 non-null   float64
 2   OHE__MathPlacement_3115   1675 non-null   float64
 3   OHE__MathPlacement_3117   1675 non-null   float64
 4   OHE__MathPlacement_3211   1675 non-null   float64
 5   OHE__MathPlacement_3213   1675 non-null   float64
 6   OHE__MathPlacement_3215   1675 non-null   float64
 7   OHE__P1_PrevGrade_F       1675 non-null   float64
 8   OHE__P1_PrevGrade_N       1675 non-null   float64
 9   remainder__HS_Gpct        1675 non-null   float64
 10  remainder__MaxMath        1675 non-null   float64
 11  remainder__MaxVerbal      1675 non-null   float64
 12  remainder__LastGPA        1675 non-null   float64
 13  remainder__Calc1Number    1675 non-null   float64
 14  remainde

In [37]:
columns_in = list(X.columns)
columns_in

['OHE__MathPlacement_3107',
 'OHE__MathPlacement_3111',
 'OHE__MathPlacement_3115',
 'OHE__MathPlacement_3117',
 'OHE__MathPlacement_3211',
 'OHE__MathPlacement_3213',
 'OHE__MathPlacement_3215',
 'OHE__P1_PrevGrade_F',
 'OHE__P1_PrevGrade_N',
 'remainder__HS_Gpct',
 'remainder__MaxMath',
 'remainder__MaxVerbal',
 'remainder__LastGPA',
 'remainder__Calc1Number',
 'remainder__Calc1LastQP',
 'remainder__PhysicsITimes']

In [38]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

y = y_m

In [39]:
y.head()

RecordNum
1    0
2    0
3    0
4    0
5    0
Name: Phys1_outcome, dtype: int64

In [40]:
import pickle

X.to_pickle('Cap_cleaned_X.pkl')
y.to_pickle("Cap_cleaned_y.pkl")